In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# This is a bit of magic to make matplotlib figures appear inline in the notebook
# rather than in a new window.

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/ifood-2019-fgvc6/sample_submission.csv
/kaggle/input/ifood-2019-fgvc6/class_list.txt
/kaggle/input/ifood-2019-fgvc6/train_set.zip
/kaggle/input/ifood-2019-fgvc6/train_labels.csv
/kaggle/input/ifood-2019-fgvc6/ifood2019_sample_submission.csv
/kaggle/input/ifood-2019-fgvc6/val_set.zip
/kaggle/input/ifood-2019-fgvc6/test_set.zip
/kaggle/input/ifood-2019-fgvc6/val_labels.csv


In [2]:
from zipfile import ZipFile
import os

if not os.path.exists("./train_set"):
    for file_name in ['train_set.zip', 'val_set.zip', 'test_set.zip']:
        with ZipFile('../input/ifood-2019-fgvc6/' + file_name, 'r') as zipObj:
            print("unzipping", file_name)
            zipObj.extractall('./')

for dirname, _, _ in os.walk('./'):
    print(dirname)

unzipping train_set.zip
unzipping val_set.zip
unzipping test_set.zip
./
./test_set
./train_set
./val_set


In [3]:
df_train = pd.read_csv('../input/ifood-2019-fgvc6/train_labels.csv')
df_val   = pd.read_csv('../input/ifood-2019-fgvc6/val_labels.csv')
df_test = pd.read_csv('../input/ifood-2019-fgvc6/sample_submission.csv')

df_train['label'] = df_train['label'].astype(str)
df_val['label'] = df_val['label'].astype(str)

train_size = df_train.shape[0]
val_size = df_val.shape[0]
test_size = df_test.shape[0]
num_classes = df_train['label'].nunique()

In [4]:
# from skimage import io
# from skimage.transform import resize

# X_train = np.zeros((10000, 256, 256, 3), dtype=np.uint8)
# X_val = np.zeros((1000, 256, 256, 3), dtype=np.uint8)

# print("load training images...")
# for row, file in zip(range(10000), df_train['img_name']):
#     if (row % 1000 == 0):
#         print(row)
#     img = io.imread("./data/train_set/" + file)
#     X_train[row] = resize(img, (256, 256))

# print("load validation images...")
# for row, file in zip(range(1000), df_val['img_name']):
#     img = io.imread("./data/val_set/" + file)
#     X_val[row] = resize(img, (256, 256))

In [5]:
from keras.preprocessing.image import ImageDataGenerator

# kernel_size = 3
# filters     = 16
# latent_dim  = 128

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

Using TensorFlow backend.


In [6]:
# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
batch_size  = 64

train_generator = train_datagen.flow_from_dataframe(
    df_train,
    x_col='img_name',
    y_col='label',    
    class_mode='categorical',
    directory='./train_set/',
    shuffle=True,
#     color_mode='grayscale',
    target_size=(256, 256),
    batch_size=batch_size)

val_generator = train_datagen.flow_from_dataframe(
    df_val,
    x_col='img_name',
    y_col='label',    
    class_mode='categorical',
    directory='./val_set/',
    shuffle=False,
#     color_mode='grayscale',
    target_size=(256, 256),
    batch_size=batch_size)

test_generator = test_datagen.flow_from_dataframe(
    df_test,
    x_col='img_name',
    directory='./test_set/',
    class_mode=None,
    shuffle=False,
#     color_mode='grayscale',
    target_size=(256, 256),
    batch_size=1)

Found 118475 validated image filenames belonging to 251 classes.
Found 11994 validated image filenames belonging to 251 classes.
Found 28377 validated image filenames.


## TPU Training

In [7]:
# from keras.layers import Dense, Activation, Flatten
# from keras.models import Sequential
# from keras.optimizers import RMSprop
# import tensorflow as tf

# # detect and init the TPU
# tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
# tf.config.experimental_connect_to_cluster(tpu)
# tf.tpu.experimental.initialize_tpu_system(tpu)

# # instantiate a distribution strategy
# tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

# # instantiating the model in the strategy scope creates the model on the TPU
# with tpu_strategy.scope():
# #     model = tf.keras.Sequential( … ) # define your model normally
# #     model.compile( … )
#     model = Sequential()
#     model.add(Flatten(input_shape=input_shape))
#     model.add(Dense(num_classes))
#     model.add(Activation('softmax'))

#     # initiate RMSprop optimizer
#     opt = RMSprop(learning_rate=0.001, decay=1e-6)

#     # Let's train the model using RMSprop
#     model.compile(loss='categorical_crossentropy',
#                   optimizer=opt,
#                   metrics=['accuracy'])

# model.fit_generator(
#     train_generator,
#     steps_per_epoch=train_size//batch_size,
#     epochs=1,
#     validation_data=val_generator,
#     validation_steps=val_size//batch_size)

## GPU training

In [8]:
# !pip uninstall tensorflow tensorflow-gpu keras --y
# !pip install tensorflow-gpu keras

# from tensorflow.python.client import device_lib
import tensorflow as tf
# import keras.backend as K

# print(device_lib.list_local_devices())
print(tf.config.experimental.list_physical_devices('GPU'))
print(tf.test.is_gpu_available())
# print(K.backend())

[]
False


### logistic regression

In [9]:

# from keras.layers import Dense, Activation, Flatten
# from keras.models import Sequential
# from keras.optimizers import RMSprop

# # network parameters
# input_shape = (256, 256, 3)
# batch_size  = 64
# epochs      = 10

# model = Sequential()
# model.add(Flatten(input_shape=input_shape))
# model.add(Dense(num_classes))
# model.add(Activation('softmax'))

# # initiate RMSprop optimizer
# opt = RMSprop(learning_rate=0.001, decay=1e-6)

# # Let's train the model using RMSprop
# model.compile(loss='categorical_crossentropy',
#               optimizer=opt,
#               metrics=['accuracy'])

# model.fit_generator(
#     train_generator,
#     steps_per_epoch=train_size//batch_size,
#     epochs=1,
#     validation_data=val_generator,
#     validation_steps=val_size//batch_size,
#     use_multiprocessing=True,
#     workers=4,
# )

## CNN

In [10]:
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Sequential
from keras.optimizers import RMSprop

# network parameters
input_shape = (256, 256, 3)

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=input_shape))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 256, 256, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 256, 256, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 254, 254, 32)      9248      
_________________________________________________________________
activation_2 (Activation)    (None, 254, 254, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 127, 127, 32)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 127, 127, 32)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 127, 127, 64)     

In [11]:
epochs      = 1

# initiate RMSprop optimizer
opt = RMSprop(learning_rate=0.001)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

model.fit_generator(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=len(val_generator),
)

Epoch 1/1
1852/1852 [==============================] - 25457s 14s/step - loss: 5.3515 - accuracy: 0.0129 - val_loss: 5.0048 - val_accuracy: 0.0328


In [12]:
# test_files = test_generator.filenames
# test_size = len(test_files)
y_predict = model.predict_generator(
    test_generator, 
    steps=test_size)
# df_test['label'] = y_predict.argmax(axis=1)
y_predict = y_predict.argsort(axis=1)[:, -3:]
df_test['label'] = [' '.join(item) for item in y_predict.astype(str)]

df_test.to_csv("submission.csv", index=False)

In [13]:
import shutil
for path in ["./train_set", "./val_set", "./test_set"]:
    shutil.rmtree(path)